In [1]:

from pyspark.sql import SparkSession

spark = (
    SparkSession.builder
    .appName("ExemploDelta")
    .config("spark.jars.packages", "io.delta:delta-spark_2.12:3.2.0")
    .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension")
    .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog")
    .getOrCreate()
)

print("Spark iniciado com Delta Lake!")


25/11/05 20:28:04 WARN Utils: Your hostname, codespaces-53bb6d resolves to a loopback address: 127.0.0.1; using 10.0.2.121 instead (on interface eth0)
25/11/05 20:28:04 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


:: loading settings :: url = jar:file:/usr/local/python/3.12.1/lib/python3.12/site-packages/pyspark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /home/codespace/.ivy2/cache
The jars for the packages stored in: /home/codespace/.ivy2/jars
io.delta#delta-spark_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-4256d2cd-0ed3-4240-aa0e-5dda8cdf1c25;1.0
	confs: [default]
	found io.delta#delta-spark_2.12;3.2.0 in central
	found io.delta#delta-storage;3.2.0 in central
	found org.antlr#antlr4-runtime;4.9.3 in central
downloading https://repo1.maven.org/maven2/io/delta/delta-spark_2.12/3.2.0/delta-spark_2.12-3.2.0.jar ...
	[SUCCESSFUL ] io.delta#delta-spark_2.12;3.2.0!delta-spark_2.12.jar (277ms)
downloading https://repo1.maven.org/maven2/io/delta/delta-storage/3.2.0/delta-storage-3.2.0.jar ...
	[SUCCESSFUL ] io.delta#delta-storage;3.2.0!delta-storage.jar (57ms)
downloading https://repo1.maven.org/maven2/org/antlr/antlr4-runtime/4.9.3/antlr4-runtime-4.9.3.jar ...
	[SUCCESSFUL ] org.antlr#antlr4-runtime;4.9.3!antlr4-runtime.jar (73ms)
:: resolution report :: resolve 1720m

Spark iniciado com Delta Lake!


In [2]:

spark.sql("CREATE DATABASE IF NOT EXISTS filmes_delta")
spark.sql("USE filmes_delta")

spark.sql("""
CREATE TABLE IF NOT EXISTS top_filmes (
    titulo STRING,
    ano INT,
    nota DOUBLE
)
USING delta
""")

print("Tabela Delta criada com sucesso!")


25/11/05 20:28:21 WARN GarbageCollectionMetrics: To enable non-built-in garbage collector(s) List(G1 Concurrent GC), users should configure it(them) to spark.eventLog.gcMetrics.youngGenerationGarbageCollectors or spark.eventLog.gcMetrics.oldGenerationGarbageCollectors


Tabela Delta criada com sucesso!


In [3]:

csv_path = "/workspaces/Apache-Spark-com-Delta-Lake-e-Apache-Iceberg/top_100_movies_full_best_effort.csv"
df = spark.read.option("header", True).csv(csv_path)
print("Dataset carregado!")
df.show(5)


Dataset carregado!
+----+--------------------+----+------------------+--------------------+--------------------+--------------------+-----------+-----------------+--------------+--------+----------+---------------+----------------+
|Rank|               Title|Year|          Genre(s)|            Director|       Main Actor(s)|             Country|IMDb Rating|Rotten Tomatoes %|Runtime (mins)|Language|Oscars Won|Box Office ($M)|Metacritic Score|
+----+--------------------+----+------------------+--------------------+--------------------+--------------------+-----------+-----------------+--------------+--------+----------+---------------+----------------+
|   1|The Shawshank Red...|1994|             Drama|      Frank Darabont|Tim Robbins|Morga...|       United States|        9.3|               91|           142| English|         0|           58.0|              82|
|   2|       The Godfather|1972|       Crime|Drama|Francis Ford Coppola|Marlon Brando|Al ...|       United States|        9.2|   

In [5]:

# Adicione o cast(Year as int) para que o schema do DataFrame
# seja compatível com o schema da tabela Delta

df.selectExpr(
    "Title as titulo",
    "cast(Year as int) as ano",  # <-- A MUDANÇA ESTÁ AQUI
    "cast(`IMDb Rating` as double) as nota"
) \
.write.format("delta").mode("overwrite").saveAsTable("filmes_delta.top_filmes")

print("Tabela Delta criada com dados iniciais!")

25/11/05 20:32:11 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


Tabela Delta criada com dados iniciais!


In [6]:

print("Exemplo SELECT:")
spark.sql("SELECT * FROM filmes_delta.top_filmes LIMIT 5").show()


Exemplo SELECT:


+--------------------+----+----+
|              titulo| ano|nota|
+--------------------+----+----+
|The Shawshank Red...|1994| 9.3|
|       The Godfather|1972| 9.2|
|     The Dark Knight|2008| 9.0|
|The Godfather: Pa...|1974| 9.0|
|        12 Angry Men|1957| 9.0|
+--------------------+----+----+



In [7]:

spark.sql("INSERT INTO filmes_delta.top_filmes VALUES ('Novo Filme', 2025, 9.4)")
print("Registro inserido!")


Registro inserido!


In [8]:

spark.sql("UPDATE filmes_delta.top_filmes SET nota = 9.9 WHERE titulo = 'Novo Filme'")
print("Registro atualizado!")


Registro atualizado!


In [9]:

spark.sql("DELETE FROM filmes_delta.top_filmes WHERE ano < 2000")
print("Registros antigos deletados!")


Registros antigos deletados!


In [10]:

print("Tabela final após operações:")
spark.sql("SELECT * FROM filmes_delta.top_filmes LIMIT 10").show()


Tabela final após operações:


+--------------------+----+----+
|              titulo| ano|nota|
+--------------------+----+----+
|     The Dark Knight|2008| 9.0|
|The Lord of the R...|2003| 8.9|
|The Lord of the R...|2001| 8.8|
|           Inception|2010| 8.8|
|The Lord of the R...|2002| 8.7|
|         City of God|2002| 8.6|
|       Spirited Away|2001| 8.6|
|            Parasite|2019| 8.6|
|        Interstellar|2014| 8.6|
|         The Pianist|2002| 8.5|
+--------------------+----+----+



In [11]:

spark.stop()
print("Execução finalizada com sucesso!")


Execução finalizada com sucesso!
